# 4 テキストを生成するための GPT モデルを一から実装する

## 4.1. LLM アーキテクチャのコーディング

**GPT** : Generative Pretrained Transformer  
パラメータ -> モデルの訓練可能な重み。

GPT-2 では 12,400,000 パラメータ (公開済み) ->  こちらの実装  
GPT-3 では 175,000,000,000 パラメータ (本の執筆時では未公開)

In [1]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,  # トークンの数 (語彙の個数)
    "context_length": 1024,  # コンテキストの長さ (入力トークンの最大数)
    "emb_dim": 768,  # 埋め込みの次元数 
    "n_heads": 12,  # Attention のヘッドの数
    "n_layers": 12,  # Transformer のブロック数
    "drop_rate": 0.1,  # ドロップアウト率 -> 0.1 は 隠れ層ユニットの 10% がランダムに無効化
    "qkv_bias": False # 現代の研究では基本無効だが、GPT-2 の重みを OpenAI から読み込むときに True にする
}

In [2]:
import torch
import torch.nn as nn

class DummyGPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
            
        self.trf_blocks = nn.Sequential(
            *[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )
        
        self.final_norm = DummyLayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )
        
    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(
            torch.arange(seq_len, device=in_idx.device)
            )
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        
        return logits
        
class DummyTransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        
    def forward(self, x):
        return x  # ダミーの実装なので何もしない

class DummyLayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
    
    def forward(self, x):
        return x  # ダミーの実装なので何もしない

**ロジット** (Rogits) : モデルの出力のこと

`DummyTransformerBlock` と `DummyLayerNorm` は後で実装する。まずは処理の流れを forward に従って実装する。

In [3]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")
batch = []
txt1 = "Every effort moves you"
txt2 = "Every day holds a"

In [4]:
print(tokenizer.encode(txt1))
print(type(tokenizer.encode(txt1)))

[6109, 3626, 6100, 345]
<class 'list'>


In [5]:
batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))

batch = torch.stack(batch, dim=0)
print(batch)
print(batch.shape)

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])
torch.Size([2, 4])


In [6]:
torch.manual_seed(123)  # 再現性のためのシード設定
model = DummyGPTModel(GPT_CONFIG_124M)
logits = model(batch)
print("Output shape:", logits.shape)
print(logits)

Output shape: torch.Size([2, 4, 50257])
tensor([[[-0.9289,  0.2748, -0.7557,  ..., -1.6070,  0.2702, -0.5888],
         [-0.4476,  0.1726,  0.5354,  ..., -0.3932,  1.5285,  0.8557],
         [ 0.5680,  1.6053, -0.2155,  ...,  1.1624,  0.1380,  0.7425],
         [ 0.0447,  2.4787, -0.8843,  ...,  1.3219, -0.0864, -0.5856]],

        [[-1.5474, -0.0542, -1.0571,  ..., -1.8061, -0.4494, -0.6747],
         [-0.8422,  0.8243, -0.1098,  ..., -0.1434,  0.2079,  1.2046],
         [ 0.1355,  1.1858, -0.1453,  ...,  0.0869, -0.1590,  0.1552],
         [ 0.1666, -0.8138,  0.2307,  ...,  2.5035, -0.3055, -0.3083]]],
       grad_fn=<UnsafeViewBackward0>)


## 4.2 層正規化を使って活性化を正規化する

**層正規化** (layer normalization)：ニューラルネットワーク層の平均 0 分散 1 になるように調整すること。  
-> 勾配の消失や爆発を防ぎ、効果的な重みへの収束を早める。

**ReLU** (Rectified Linear Unit) : 負の値を 0 にするだけ

In [7]:
torch.manual_seed(123)  # 再現性のためのシード設定
batch_example = torch.randn(2, 5) # ダミーのバッチデータ　5つの次元 (特徴量) をもつ、2 つの訓練サンプル
print(batch_example, "\n")
layer = nn.Sequential(nn.Linear(5, 6), nn.ReLU()) # 5 -> 6次元線形変換する層を経た後、ReLU 活性化関数を適用する層の実装
out = layer(batch_example)
print(out)

tensor([[-0.1115,  0.1204, -0.3696, -0.2404, -1.1969],
        [ 0.2093, -0.9724, -0.7550,  0.3239, -0.1085]]) 

tensor([[0.2260, 0.3470, 0.0000, 0.2216, 0.0000, 0.0000],
        [0.2133, 0.2394, 0.0000, 0.5198, 0.3297, 0.0000]],
       grad_fn=<ReluBackward0>)


各行平均と分散を計算

In [8]:
mean = out.mean(dim=1, keepdim=True)
var = out.var(dim=1, keepdim=True)
print("Mean:\n", mean)
print("Variance:\n", var)

Mean:
 tensor([[0.1324],
        [0.2170]], grad_fn=<MeanBackward1>)
Variance:
 tensor([[0.0231],
        [0.0398]], grad_fn=<VarBackward0>)


`keepdim=True`: `dim` で指定した次元に沿って平均を計算する。指定しない場合は次のようになる。

In [9]:
mean_ = out.mean(dim=1)
var_ = out.var(dim=1)
print("Mean:\n", mean_)
print("Variance:\n", var_)

Mean:
 tensor([0.1324, 0.2170], grad_fn=<MeanBackward1>)
Variance:
 tensor([0.0231, 0.0398], grad_fn=<VarBackward0>)


**標準化** (normarizetion) : 平均で引いて分散で割る  

層の出力に層正規化を適用する。

In [10]:
out_norm = (out - mean) / torch.sqrt(var)
print("Normalized layer outputs:\n", out_norm)

mean = out_norm.mean(dim=-1, keepdim=True)
var = out_norm.var(dim=-1, keepdim=True)

print("Mean:\n", mean)
print("Variance:\n", var)

Normalized layer outputs:
 tensor([[ 0.6159,  1.4126, -0.8719,  0.5872, -0.8719, -0.8719],
        [-0.0189,  0.1121, -1.0876,  1.5173,  0.5647, -1.0876]],
       grad_fn=<DivBackward0>)
Mean:
 tensor([[9.9341e-09],
        [1.9868e-08]], grad_fn=<MeanBackward1>)
Variance:
 tensor([[1.0000],
        [1.0000]], grad_fn=<VarBackward0>)


科学表記 (`e-09`) とかをオフにできる

In [12]:
torch.set_printoptions(sci_mode=False)
print("Mean:\n", mean)
print("Variance:\n", var)

Mean:
 tensor([[    0.0000],
        [    0.0000]], grad_fn=<MeanBackward1>)
Variance:
 tensor([[1.0000],
        [1.0000]], grad_fn=<VarBackward0>)


In [14]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim, eps=1e-5):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))
    
    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False) # 不変ではなく有偏分散。n がデカいのでその差がほぼ無視でき、正規化層との互換性を保つため。
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

- `self.scale` : 入力と同じ次元。訓練時に処理するデータのスケーリングを学習。
- `self.shift` : 入力と同じ次元。訓練時に処理するデータのシフトを学習

層を介して適切な処理を行う

In [17]:
ln = LayerNorm(emb_dim=5)
out_ln = ln(batch_example)
mean = out_ln.mean(dim=-1, keepdim=True)
var = out_ln.var(dim=-1, keepdim=True, unbiased=False)

print("Mean:\n", mean)
print("Variance:\n", var)

Mean:
 tensor([[    -0.0000],
        [     0.0000]], grad_fn=<MeanBackward1>)
Variance:
 tensor([[1.0000],
        [1.0000]], grad_fn=<VarBackward0>)
